In [ ]:
import pandas as pd
import itertools
from ortools.linear_solver import pywraplp as OR

In [ ]:
ex = pd.read_csv('fws_7_students.csv', index_col=0)

In [ ]:
def fws(preferences, capcity, cost, integer = False):
    """A model for solving a first-year writing seminar assignment problem.
    
    Args:
        preferences (pd.DataFrame): Preferred classes for each student.
        capcity (int): Capacity of the classroom.
        cost (Dict): Dictionary from edges types to cost.
    """
    STUDENTS = list(preferences.index) + ['dummy']             # students
    
    CLASSES = []
    for c in preferences.columns:
        CLASSES = CLASSES + list(preferences[c].unique())
    CLASSES = list(set(CLASSES))                               # classes
    
    edge_costs = {}                                 
    for s in preferences.index:
        for c in preferences:
            edge_costs[(s, preferences.at[int(s),c])] = cost[int(c)]
    EDGES = list(edge_costs)                                   # edges
    
    # add dummy edges
    dummy_edges = list(itertools.product(['dummy'], CLASSES))
    EDGES.extend(dummy_edges)          
    
    c = edge_costs                                             # costs
    # add dummy edge costs
    for edge in dummy_edges:
        c.update({edge : cost['dummy']})    
    
    # define model
    m = OR.Solver('fws', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        if integer:
            x[i,j] = m.IntVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
        else:
            x[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
        
    # objective function
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
       
    # subject to: each student (not including the dummy) is assigned at most one class
    for k in STUDENTS:
        if k != 'dummy':
            m.Add(sum(x[i,j] for i,j in EDGES if i==k) <= 1)
        
    # subject to: each class is full
    for k in CLASSES:
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) == capcity)
    
    return m,x

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
costs = {1:1, 2:2, 'dummy':3}
m,x = fws(ex, 2, costs, integer=True)
solve(m)